# Llama2 LangChain Output Parsers  

 > Fixing common errors from Llama2 outputs that do not conform to standards set by OpenAI. 

In [ ]:
#| default_exp utils.output_parsers
from nbdev import *

In [ ]:
#| export
import asyncio

import json

from langchain.callbacks.base import BaseCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import AgentAction, AgentFinish, HumanMessage, AIMessage
from langchain.schema import BaseMessage, BaseOutputParser, OutputParserException
from langchain.schema import LLMResult
from langchain.utils.input import print_text

import os
import re
import sys
from typing import (
    Any,
    Dict,
    List,
    Union,
    Optional
)

import json
import logging
import re
from typing import Union

from langchain.agents.agent import AgentOutputParser
from langchain.agents.chat.prompt import FORMAT_INSTRUCTIONS
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.output_parsers.json import parse_json_markdown



In [ ]:
#| export

class JsonEnclosedByTextOutputParser(BaseOutputParser[Any]):
    """Parse the output of an LLM call to a JSON object."""

    def parse(self, text: str) -> Any:
        text = text.strip()
        fixed = re.sub(r'(?<!\\)\\(?!["\\/bfnrt]|u[0-9a-fA-F]{4})', r'', text)
        m = re.search('.*([\[\{](.|\n)*[\}\]]).*', fixed, flags=re.M)
        
        if m:
            text1 = m.group(1)

            try:
                return json.loads(text1)
            except json.JSONDecodeError as e:
                try:
                    # need to make sure all entries in the JSON are quoted
                    # so that the JSON parser can parse it
                    # e.g. {"a": E} -> {"a": "E"}
                    # this is a hack, but it works
                    text2 = re.sub(r':\s*([a-zA-Z0-9_]+)\s*([,\]\}])', r': "\1"\2', text1)
                except json.JSONDecodeError as e1:
                    raise OutputParserException(f"Invalid json output: {text2} \n\nderived from\n\n {text}") from e1
        else: 
            raise OutputParserException(f"Could not find json-formatted data in: {text}")

    @property
    def _type(self) -> str:
        return "json_enclosed_by_text_output_parser"
    


NameError: name 'BaseOutputParser' is not defined

In [ ]:
#| export

class MermaidExtractionOutputParser(BaseOutputParser[Any]):
    """Parse the output of an LLM call to a JSON object."""

    def parse(self, text: str) -> Any:
        text = text.strip()
        m = re.search('.*(```mermaid:.*graph .* ```).*', text, flags=re.M)
        if m:
            text1 = m.group(1)
            return text1
        else: 
            raise OutputParserException(f"Could not find mermaid-formatted data in: {text}")

    @property
    def _type(self) -> str:
        return "json_enclosed_by_text_output_parser"
    
